In [2]:
import os
import yaml

import numpy as np
import pandas as pd

pd.options.display.max_rows = 300

In [3]:
artifacts_fpath = "/local-scratch/nigam/projects/lguo/temp_ds_shift_robustness/experiments/dg/artifacts"
hparams_fpath = "/local-scratch/nigam/projects/lguo/temp_ds_shift_robustness/experiments/dg/hyperparams"
tasks = {
    'hospital_mortality':'In-Hospital Mortality',
    'LOS_7':'LOS > 7 Days',
    'readmission_30':'Readmission in 30 Days',
    'icu_admission':'ICU Admission'
}
algos = {
    'irm':'IRM',
    'dro':'GroupDRO',
    'coral':'CORAL',
    'adversarial':'AL',
}
years = {
    '2009_2010_2011_2012':'09-12',
    '2010_2011_2012_2013':'10-13',
    '2011_2012_2013_2014':'11-14',
    '2012_2013_2014_2015':'12-15',
    '2013_2014_2015_2016':'13-16',
    '2014_2015_2016_2017':'14-17',
    '2015_2016_2017_2018':'15-18',
    '2009_2010_2011_2012_2013':'09-13',
    '2009_2010_2011_2012_2013_2014':'09-14',
    '2009_2010_2011_2012_2013_2014_2015':'09-15',
    '2009_2010_2011_2012_2013_2014_2015_2016':'09-16',
    '2009_2010_2011_2012_2013_2014_2015_2016_2017':'09-17',
    '2009_2010_2011_2012_2013_2014_2015_2016_2017_2018':'09-18',
}

#### Grid

In [4]:
df_hparams = pd.DataFrame()

for algo in algos:
    fpath = os.path.join(
        hparams_fpath,
        f"{algo}.yml"
    )
    hparams = yaml.load(
        open(f"{fpath}"),
        Loader=yaml.FullLoader
    )
    
    if algo=='dro':
        lambda_var = 'lr_lambda'
    else:
        lambda_var = 'lambda_group_regularization'
    
    df_hparams = pd.concat((
        df_hparams,
        pd.DataFrame({
            'Algorithm':[algos[algo]],
            'Lambda':[','.join([str(x) for x in hparams[lambda_var]])],
        })
    ))

In [5]:
df_hparams

,Algorithm,Lambda
0,IRM,"0.1,1,10,100,1000,10000"
0,GroupDRO,"0.1,0.01,0.001,0.0001,1e-05,1e-06"
0,CORAL,"0.01,0.1,1,10,100,1000"
0,AL,"1,0.1,0.01,0.001,0.0001,1e-05"


#### Selected

In [6]:
df_hparams = pd.DataFrame()

for task in tasks:
    for algo in algos:
        for year in years:

            fpath = os.path.join(
                artifacts_fpath,
                task,
                f"models/{algo}_{year}"
            )

            best_model_name = [x for x in os.listdir(fpath) if "best_model" in x][0]

            hparams = yaml.load(
                open(f"{fpath}/{best_model_name}/hparams.yml"),
                Loader=yaml.FullLoader
            )

            df_hparams = pd.concat((
                df_hparams,
                pd.DataFrame({k:[v] for k,v in hparams.items()}).assign(
                    Task=tasks[task],
                    Train_Group=years[year],
                    Algorithm=algos[algo],
                    Lambda=hparams['lr_lambda'] if algo=='dro' else hparams['lambda_group_regularization']
                )
            ))
df_hparams = df_hparams[['Task','Algorithm','Train_Group','Lambda']]

In [7]:
df_hparams.query("Train_Group=='09-12'")

,Task,Algorithm,Train_Group,Lambda
0,In-Hospital Mortality,IRM,09-12,1.000000
0,In-Hospital Mortality,GroupDRO,09-12,0.000100
0,In-Hospital Mortality,CORAL,09-12,1.000000
0,In-Hospital Mortality,AL,09-12,0.000010
0,LOS > 7 Days,IRM,09-12,0.100000
0,LOS > 7 Days,GroupDRO,09-12,0.000100
0,LOS > 7 Days,CORAL,09-12,0.010000
0,LOS > 7 Days,AL,09-12,0.000010
0,Readmission in 30 Days,IRM,09-12,1.000000
0,Readmission in 30 Days,GroupDRO,09-12,0.000001


In [8]:
df_hparams

,Task,Algorithm,Train_Group,Lambda
0,In-Hospital Mortality,IRM,09-12,1.000000
0,In-Hospital Mortality,IRM,10-13,0.100000
0,In-Hospital Mortality,IRM,11-14,0.100000
0,In-Hospital Mortality,IRM,12-15,0.100000
0,In-Hospital Mortality,IRM,13-16,0.100000
0,In-Hospital Mortality,IRM,14-17,1.000000
0,In-Hospital Mortality,IRM,15-18,0.100000
0,In-Hospital Mortality,IRM,09-13,0.100000
0,In-Hospital Mortality,IRM,09-14,0.100000
0,In-Hospital Mortality,IRM,09-15,0.100000
